In [1]:
!pip install opencv-python

Подключаем диск

In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


Импортируем библиотеки

In [4]:
import tarfile
import pandas as pd
import zipfile
from zipfile import ZipFile
import os
from os import listdir
import cv2
import numpy as np
from tqdm import tqdm
import time
from tempfile import TemporaryDirectory

Загружаем данные

In [5]:
!cp /content/drive/MyDrive/data/camvid.zip .

file_name = "camvid.zip"
with ZipFile(file_name, 'r') as zip:

    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

!ls CamVid

Extracting all the files now...
Done!
class_dict.csv	test  test_labels  train  train_labels	val  val_labels


In [8]:
# классы, представленные в датасете
df_class = pd.read_csv("./CamVid/class_dict.csv")

Сделаем df_train, df_val и df_test, в которых будут храниться пути до изображений и соответствующих им масок

In [6]:
dir_path = "./CamVid/"

train_images = listdir("./CamVid/train")
val_images = listdir("./CamVid/val")
test_images = listdir("./CamVid/test")

dict_train = {
    "img_path": [dir_path + "train/" + img_name for img_name in train_images],
    "mask_path": [dir_path + "train_labels/" + img_name.split(".")[0] + "_L.png" for img_name in train_images]
}
dict_val = {
    "img_path": [dir_path + "val/" + img_name for img_name in val_images],
    "mask_path": [dir_path + "val_labels/" + img_name.split(".")[0] + "_L.png" for img_name in val_images]
}
dict_test = {
    "img_path": [dir_path + "test/" + img_name for img_name in test_images],
    "mask_path": [dir_path + "test_labels/" + img_name.split(".")[0] + "_L.png" for img_name in test_images]
}

df_train = pd.DataFrame(dict_train)
df_val = pd.DataFrame(dict_val)
df_test = pd.DataFrame(dict_test)

In [7]:
!ls

CamVid	camvid.zip  drive  sample_data


Создадим дирректорию, в которой будем хранить преобразованные маски

In [10]:
os.makedirs("labels_np")

In [12]:
!ls

CamVid	camvid.zip  drive  labels_np  sample_data


Предварительно сгенерируем закодированные маски для каждого изображения и сохраним их

In [14]:
# генерация масок

cls2idx = {}
idx2cls = {}
idx2rgb = {}
for i in range(len(df_class)):
  label = df_class.values[i][0]
  arr_rgb = df_class.values[i][1:]
  idx = i
  cls2idx[label] = idx
  idx2cls[idx] = label
  idx2rgb[label] = arr_rgb



def make_np_masks(df, dir_path):

  new_masks_path_lst = []

  try:
    os.makedirs(dir_path)
  except FileExistsError:
    print("directory exists")
    return

  for i in tqdm(range(len(df))):
    mask_path = df.iloc[i].mask_path
    mask = cv2.imread(mask_path)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
    res_mask = []
    for colour in list(idx2rgb.values()):
        equality = np.equal(mask, colour)
        class_map = np.all(equality, axis = -1)
        res_mask.append(class_map)
    res_mask = np.uint8(np.stack(res_mask, axis=-1))

    mask_name = (mask_path.split("/")[-1]).split(".")[0]
    save_path = dir_path + mask_name + ".npy"

    with open(save_path, "wb") as f:
      np.save(save_path, res_mask)

    new_masks_path_lst.append(save_path)

  return new_masks_path_lst


# make_np_masks(df_val, "./CamVid/val_labels_np/")

train_res_paths = make_np_masks(df_train, "./labels_np/train_labels_np/")
val_res_paths = make_np_masks(df_val, "./labels_np/val_labels_np/")
test_res_paths = make_np_masks(df_test, "./labels_np/test_labels_np/")

100%|██████████| 232/232 [11:53<00:00,  3.07s/it]


Архивируем полученные маски в npy формате и скачаем

In [15]:
import tarfile
import os.path

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))


In [17]:
make_tarfile("labels_np.tar.gz", './labels_np/')

Загружаем полученный файл

In [19]:
!ls /content/drive/MyDrive/data

256_ObjectCategories.tar  car_object_detection.zip  hymenoptera_data.zip
camvid.zip		  HAM10000_metadata.csv


In [21]:
!cp labels_np.tar.gz /content/drive/MyDrive/data/

In [22]:
!ls /content/drive/MyDrive/data

256_ObjectCategories.tar  car_object_detection.zip  hymenoptera_data.zip
camvid.zip		  HAM10000_metadata.csv     labels_np.tar.gz
